In [ ]:
import os
from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)

print(chat_completion.choices[0].message.content)

In [7]:
from typing import Literal

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# pip install langgraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

# 定义工具函数，用于代理调用外部工具
@tool
def search(query: str):
    """模拟一个搜索工具"""
    if "上海" in query.lower() or "Shanghai" in query.lower():
        return "现在30度，有雾。"
    return "现在是35度，阳光明媚。"

# 将工具的函数放入工具列表
tools = [search]

# 创建工具节点
tool_node = ToolNode(tools)

# 初始化模型和工具，定义并绑定工具到模型
model = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(tools)
# 定义函数，决定是否继续执行
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # 如果LLM调用了工具，则转到“tools”节点
    if last_message.tool_calls:
        return "tools"
    # 否则，停止（回复用户）
    return END

# 定义调用模型的函数
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # 返回列表，因为这将被添加到现有列表中
    return {"messages": [response]}

# 2. 用状态初始化图，定义一个新的状态图
workflow = StateGraph(MessagesState)

# 3. 定义图节点，定义我们将循环的两个节点
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 4. 定义入口点和图边
# 设置入口点为“agent”
# 这意味着这是第一个被调用的节点
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)

# 6. 执行图，使用可运行对象
final_state = app.invoke(
    input={"messages": [HumanMessage(content="上海的天气怎么样?")]},
    config={"configurable": {"thread_id": 42}}
)
# 从 final_state 中获取最后一条消息的内容
result = final_state["messages"][-1].content
print(result)

final_state = app.invoke(
    input={"messages": [HumanMessage(content="我问的那个城市?")]},
    config={"configurable": {"thread_id": 42}}
)
result = final_state["messages"][-1].content
print(result)

# 将生成的图片保存到文件
graph_png = app.get_graph().draw_mermaid_png()
with open("langgraph_hello.png", "wb") as f:
    f.write(graph_png)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

### 核心组件

In [5]:
# 从langgraph.graph模块导入START和StateGraph
from langgraph.graph import START, StateGraph, END

# 定义一个节点函数my_node，接收全局状态和配置，返回新的状态
def my_node(state, config):
    return {"x": state["x"] + 1, "y": state["y"] + 2}

# 创建一个状态图构建器builder，使用字典类型作为状态类型
builder = StateGraph(dict)

# 向构建器中添加节点my_node，节点名称将自动设置为'my_node'
builder.add_node(my_node)  # node name will be 'my_node'

# 添加一条边，从START到'my_node'节点
builder.add_edge(START, "my_node")
builder.add_edge("my_node", END)
# 编译状态图，生成可执行的图
graph = builder.compile()

# 调用编译后的图，传入初始状态{"x": 1, "y": 2}
print(graph.invoke({"x": 1, "y": 5}))

{'x': 2, 'y': 7}
